## Imports and auxiliar functions

In [18]:
# !pip install pandas
# !pip install scikit-learn
# !pip install statsmodels
# !pip install openpyxl

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit,GridSearchCV

from statsmodels.tsa.ar_model import AutoReg
from sklearn.linear_model import Lasso, Ridge, ElasticNet
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, RandomForestRegressor

from sklearn.metrics import root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error


# Setting a random state
np.random.seed(42)



# Create lagged features
def create_lagged_columns(df, list_cols, list_lags, df_with_target=None):
    """Create lagged covariates for time series forecasting."""
    lagged_data = {}
    for column in list_cols:
        for lag in list_lags:
            if df_with_target is None:
                lagged_data[f"{column}_lag{lag}"] = df[column].shift(lag)
            else:
                lagged_data[f"{column}_lag{lag}"] = df_with_target[column].shift(lag)

    lagged_df = pd.DataFrame(data=lagged_data, index=df.index)

    return lagged_df



def walk_foward_validation(model, param_grid, X_train, y_train):
    tscv = TimeSeriesSplit(n_splits=2, test_size=1)  # changed from n_splits=12 to n_splits=2 due to time complexity

    # GridSearchCV to find the best hyperparameters using Walk-Forward CV
    search = GridSearchCV(model, param_grid, cv=tscv, scoring='neg_root_mean_squared_error')

    # Perform the grid search over the training set
    search.fit(X_train, y_train.values.ravel())

    # Output the best parameters from the grid search
    print(f"Best Estimator: {search.best_estimator_}")

    return search.best_estimator_



def recursive_forecast(model, y_train, X_train, X_test, forecast_horizon, target_variable, lags_target, cv=False):
    predictions = []
    y_train_step = y_train.copy()

    if X_test is None:
        for i in range(forecast_horizon):
            model_func = model(y_train=y_train_step)
            model_fitted = model_func.fit()

            # Predict the next step
            y_pred = model_fitted.predict(start=len(y_train_step), end=len(y_train_step))
            predictions.append(y_pred.iloc[0])

            new_row_prediction = y_pred.to_frame(name=y_train_step.columns[0])
            y_train_step = pd.concat([y_train_step, new_row_prediction])

    else:
        model_func = model(X_train, y_train_step)
        model_fitted = model_func.fit(X=X_train, y=y_train.values.ravel())

        for i in range(forecast_horizon):
            X_test_step = X_test.iloc[i:i+1]  # one step ahead forecast

            lag_target_columns = [f"{target_variable}_lag{lag}" for lag in lags_target]
            lag_target_values = y_train_step.iloc[[-lag for lag in lags_target]].values.flatten().reshape(1,-1)
            X_test_lag = pd.DataFrame(data=lag_target_values, columns=lag_target_columns, index=X_test_step.index)

            X_test_for_pred = pd.concat([X_test_step, X_test_lag], axis=1)

            # Predict the next step
            y_pred = model_fitted.predict(X=X_test_for_pred)[0]
            predictions.append(y_pred)

            # Append the new row with the prediction to the training data
            new_row_prediction = pd.DataFrame({target_variable: [y_pred]}, index=X_test_step.index)
            y_train_step = pd.concat([y_train_step, new_row_prediction])

    return np.array(predictions)


## Model Functions

In [35]:

# 1. Random Walk (RW)
def random_walk(y_train, forecast_horizon):
    last_observation = y_train.iloc[-1].values[0]
    return np.array([last_observation] * forecast_horizon)

# 2. Autoregressive (AR)
def autoregressive(y_train):
    y_train = y_train.asfreq('MS')
    model = AutoReg(y_train, lags=12)
    return model

# 3. LASSO Regression
def lasso(X_train, y_train):
    model = Lasso(max_iter=10000)
    param_grid = {
        'alpha': [0.0001, 0.0003, 0.0006, 0.001, 0.003, 0.006, 0.01, 0.03, 0.06, 0.1, 0.3, 0.6, 1, 3, 6]
    }
    best_model = walk_foward_validation(model=model, param_grid=param_grid, X_train=X_train, y_train=y_train)
    return best_model

# 4. Ridge Regression
def ridge(X_train, y_train):
    model = Ridge(max_iter=10000)
    param_grid = {
        'alpha': [0.1, 0.3, 0.6, 1, 3, 6, 10, 13, 16, 20, 30, 40, 50, 60, 70, 80, 90, 100, 150, 200, 300]
    }
    best_model = walk_foward_validation(model=model, param_grid=param_grid, X_train=X_train, y_train=y_train)
    return best_model

# 5. Elastic Net (Elnet)
def elastic_net(X_train, y_train):
    model = ElasticNet(max_iter=40000)
    param_grid = {
        'alpha': np.arange(0.01, 0.52, 0.05),
        'l1_ratio': np.arange(0.01, 0.52, 0.05)
    }
    best_model = walk_foward_validation(model=model, param_grid=param_grid, X_train=X_train, y_train=y_train)
    return best_model

# 6. Bagging (Bootstrap Aggregating)
def bagging(X_train, y_train):
    model = BaggingRegressor()
    param_grid = {
        'n_estimators': np.arange(150, 350, 50),            # Number of base models in ensemble
        'max_features': np.arange(0.1, 1, 0.1),             # Proportion of features used to train each base model
        'bootstrap': [False],                               # Do not use bootstrap samples
        'bootstrap_features': [True],                       # Use bootstrap features
    }
    best_model = walk_foward_validation(model=model, param_grid=param_grid, X_train=X_train, y_train=y_train)
    return best_model

# 7. Gradient Boosting
def gradient_boosting(X_train, y_train):
    model = GradientBoostingRegressor()
    param_grid = {
        'n_estimators': np.arange(200, 400, 50),            # Number of boosting stages
        'learning_rate': np.arange(0.01, 0.2, 0.05),        # Learning rate between 0.01 and 0.2
        'max_depth': np.arange(3, 20, 2),                   # Maximum depth of trees
        'min_samples_split': np.arange(6, 20, 2),           # Minimum number of samples to split a node
        'min_samples_leaf': np.arange(1, 4, 1),             # Minimum number of samples at a leaf node
        'subsample': np.arange(0.8, 1.0, 0.05)              # Fraction of samples used for fitting each tree
    }
    best_model = walk_foward_validation(model=model, param_grid=param_grid, X_train=X_train, y_train=y_train)
    return best_model

# 8. Random Forest
def random_forest(X_train, y_train):
    model = RandomForestRegressor()
    param_grid = {
        'n_estimators': np.arange(100, 300, 50),            # Number of trees
        'max_features': np.arange(0.3, 1.0, 0.05),          # Maximum features used in splits
        'bootstrap': [False],                               # Do not use bootstrap samples
    }
    best_model = walk_foward_validation(model=model, param_grid=param_grid, X_train=X_train, y_train=y_train)
    return best_model


## Workflow functions

In [36]:

def bronze_to_silver(df):
    # Ensure that modifications on one DataFrame do not affect the other
    df = df.copy()

    # Drop the first row, it was a dummy one
    df = df.iloc[1:]

    # Set the column 'date' as the index of the DataFrame
    df = df.set_index('date').sort_index()

    # Convert 'date' index to datetime if it's not already
    df.index = pd.to_datetime(df.index)
    
    # Filter rows based on date range
    df = df[(df.index >= '2000-01-01') & (df.index <= '2023-12-01')]

    # Replace entries containing any of the keywords with NaN and remove columns with all entries as NaN
    keywords = ['european', 'euro', 'germany', 'uk', ':']
    for column in df.columns:
        df[column] = df[column].apply(lambda x: np.nan if isinstance(x, str) and any(keyword in x.lower() for keyword in keywords) else x)
        if df[column].isnull().all():
            df.drop(column, axis=1, inplace=True)

    return df



def silver_to_gold(df):
    # Ensure that modifications on one DataFrame do not affect the other
    df = df.copy()

    # Drop columns with all NaNs
    df = df.dropna(axis=1)

    # for each variable, keep the column with the highest degree of adjustment
    # after droping na's to unsure that, in the end, atleast one viable variable is available
    adjustment_degree = {"SCA": 1, "SA": 2, "CA": 3, "NSA": 4, "NA": 5}
    dict_cols = {}
    final_cols = []

    for column in df.columns:
        if column.startswith("HICP"):
            if not column.startswith("HICP_All_"):  # HICP_All_idx, HICP_All_mor, HICP_All_anr
                continue
        parts = column.rsplit('_', 1)
        key = parts[0]
        suffix = parts[1]
        value = (suffix, column)
        
        if key not in dict_cols:
            dict_cols[key] = value
        else:
            current_suffix = dict_cols[key][0]
            if suffix not in adjustment_degree:
                print(f"the suffix '{suffix}' is not a valid adjustment")
                continue
            if adjustment_degree[suffix] < adjustment_degree[current_suffix]:
                dict_cols[key] = value

    for key, value in dict_cols.items():
        final_cols.append(value[1])
    
    # Filter columns to include only the selected ones
    df = df[final_cols]

    return df



def gold_to_model_data(df, target_variable, list_target_variables, test_size, list_lags_covariates, list_lags_target):
    # Split the data into X_train, X_test, y_train, y_test by the target variable and the test size and standardize it

    # Splitting target variable 'y'
    # y is either "HICP_All_idx_NA", "HICP_All_mor_NA", "HICP_All_anr_NA"
    y = df[[target_variable]]
    y_train = y[:-test_size]
    y_test = y[-test_size:]

    # Standardizing 'y'
    scaler_y = StandardScaler()
    scaler_y_fitted = scaler_y.fit(y_train)
    y_train_scaled = pd.DataFrame(scaler_y_fitted.transform(y_train), columns=y_train.columns, index=y_train.index)
    y_test_scaled = pd.DataFrame(scaler_y_fitted.transform(y_test), columns=y_test.columns, index=y_test.index)

    # Splitting feature set 'X'
    X = df.drop(list_target_variables, axis=1)
    X_train = X[:-test_size]
    X_test = X[-test_size:]

    # Standardizing 'X'
    scaler_X = StandardScaler()
    scaler_X_fitted = scaler_X.fit(X_train)
    X_scaled = pd.DataFrame(scaler_X_fitted.transform(X), columns=X.columns, index=X.index)

    # Creating lagged covariate features
    X_lagged_covariates = create_lagged_columns(df=X_scaled, list_cols=X_scaled.columns, list_lags=list_lags_covariates)
    X_scaled = pd.concat([X_scaled, X_lagged_covariates], axis=1)

    # Creating lagged target features (appended only to the training set)
    X_train_scaled = pd.DataFrame(X_scaled[:-test_size], columns=X_scaled.columns, index=X_train.index)
    X_train_lagged_target = create_lagged_columns(df=X_train_scaled, list_cols=[target_variable], list_lags=list_lags_target, df_with_target=y_train_scaled)
    X_train_scaled = pd.concat([X_train_scaled, X_train_lagged_target], axis=1)

    # Preparing test set
    X_test_scaled = pd.DataFrame(X_scaled[-test_size:], columns=X_scaled.columns, index=X_test.index)

    # Aligning the training dataset
    df_training_scaled_alignment = pd.concat([y_train_scaled, X_train_scaled], axis=1).dropna()
    y_train_scaled_align = df_training_scaled_alignment[y_train_scaled.columns]
    X_train_scaled_align = df_training_scaled_alignment[X_train_scaled.columns]

    return X_train_scaled_align, X_test_scaled, y_train_scaled_align, y_test_scaled, scaler_X_fitted, scaler_y_fitted



def model_data_to_model_evaluation(target_variable, model_func, X_train, X_test, y_train, y_test, scaler_X_fitted, scaler_y_fitted, forecast_horizon, list_lags_target):
    evaluation = {}

    if model_func.__name__ in ['random_walk']:
        # Perform forecast for the horizon period
        y_pred = model_func(y_train, forecast_horizon)
    
    elif model_func.__name__ in ['autoregressive']:
        # Perform recursive forecast for the horizon period
        y_pred = recursive_forecast(
            model=model_func, y_train=y_train, X_train=None, X_test=None, forecast_horizon=forecast_horizon,
            target_variable=target_variable, lags_target=list_lags_target
        )

    elif model_func.__name__ in ['lasso', 'ridge', 'elastic_net', 'bagging', 'gradient_boosting', 'random_forest']:
        # Perform recursive forecast for the horizon period
        y_pred = recursive_forecast(
            model=model_func, y_train=y_train, X_train=X_train, X_test=X_test, forecast_horizon=forecast_horizon,
            target_variable=target_variable, lags_target=list_lags_target
        )

    else:
        print(f"Did not handle the model: {model_func.__name__}")

    y_test_for_horizon = y_test[:forecast_horizon]

    y_test_inversed = scaler_y_fitted.inverse_transform(np.array(y_test_for_horizon).reshape(-1, 1))
    y_pred_inversed = scaler_y_fitted.inverse_transform(np.array(y_pred).reshape(-1, 1))

    # Evaluate the model
    rmse = root_mean_squared_error(y_test_inversed, y_pred_inversed)  # Root Mean Squared Error
    mae = mean_absolute_error(y_test_inversed, y_pred_inversed)  # Mean Absolute Error
    mape = mean_absolute_percentage_error(y_test_inversed, y_pred_inversed)  # Mean Absolute Percentage Error

    # Store the results in the dictionary
    evaluation = {
        'rmse': round(rmse, 4),
        'mae': round(mae, 4),
        'mape': round(mape, 4),
    }

    return evaluation



def workflow_results(excel_path, list_sheet_name, list_model, test_size, forecast_horizon, list_lags_covariates, list_lags_target):
    # Load the Excel file
    xls = pd.ExcelFile(excel_path)

    # Dictionary to store the results by sheet_name, by target variable and then group them by models
    dict_results = {}

    # Iterate through each sheet name to process bronze to silver and silver to gold transformations
    for sheet_name in list_sheet_name:
        print("sheet_name:", sheet_name)

        # Initialize the dictionary to store results for the current sheet_name
        if sheet_name not in dict_results:
            dict_results[sheet_name] = {}

        # Read the sheet into a dataframe
        bronze_df = pd.read_excel(xls, sheet_name=sheet_name)

        # Apply the bronze_to_silver transformation
        silver_df = bronze_to_silver(df=bronze_df)

        # Apply the silver_to_gold transformation
        gold_df = silver_to_gold(df=silver_df)

        list_target_variables = []
        for column in gold_df.columns:
            if column.startswith("HICP"):
                list_target_variables.append(column)

        for target_variable in list_target_variables:
            print("    target:", target_variable)

            # Initialize the dictionary to store results for the current target_variable in the target_variable sheet_name
            if target_variable not in dict_results[sheet_name]:
                dict_results[sheet_name][target_variable] = {}

            X_train, X_test, y_train, y_test, scaler_X_fitted, scaler_y_fitted = gold_to_model_data(
                df=gold_df, target_variable=target_variable, list_target_variables=list_target_variables, test_size=test_size,
                list_lags_covariates=list_lags_covariates, list_lags_target=list_lags_target
            )

            # Apply each model function to the current target_variable from the current sheet_name's data
            for model_func in list_model:
                print("        model:", model_func.__name__)

                # Initialize the dictionary to store results for the current target_variable in the target_variable sheet_name
                if model_func.__name__ not in dict_results[sheet_name][target_variable]:
                    dict_results[sheet_name][target_variable][model_func.__name__] = {}

                # Apply the predictive model to obtain its evaluation
                model_evaluation = model_data_to_model_evaluation(
                    target_variable=target_variable, model_func=model_func,
                    X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,
                    scaler_X_fitted=scaler_X_fitted, scaler_y_fitted=scaler_y_fitted,
                    forecast_horizon=forecast_horizon, list_lags_target=list_lags_target
                )

                # Store the model evaluation metrics under the corresponding model name
                dict_results[sheet_name][target_variable][model_func.__name__] = model_evaluation
        print()
    return dict_results


## Initiate workflow

In [ ]:

# Path to the Excel file
folder_path = "C:/Users/put/your/folder/path/that/points/to/file"
excel_file = "MFW_VascoCostaLeal_EUED_dataset.xlsx"
excel_path = f"{folder_path}/{excel_file}"

# Setting variables
list_sheet_name = ["EU27", "EA20", "Germany", "UK"]
list_model = [
    random_walk, autoregressive,
    lasso, ridge, elastic_net,
    bagging, gradient_boosting, random_forest,
]

test_size = 12
forecast_horizon = 12
list_lags_covariates = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
list_lags_target = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


results = workflow_results(
    excel_path=excel_path, list_sheet_name=list_sheet_name, list_model=list_model,
    test_size=test_size, forecast_horizon=forecast_horizon, 
    list_lags_covariates=list_lags_covariates, list_lags_target=list_lags_target
)


In [ ]:
# print(results)
# print(results["EU27"])
# print(results["EU27"]["HICP_All_mor_NA"])
# print(results["EU27"]["HICP_All_mor_NA"]["linear_regression"])
# print(results["EU27"]["HICP_All_mor_NA"]["linear_regression"]["rmse"])


# Flattening the results into a tabular format
table_data = []

for region, targets in results.items():
    for target_variable, models in targets.items():
        for model_name, metrics in models.items():
            table_data.append([region, target_variable, model_name, metrics['rmse'], metrics['mae']])

df = pd.DataFrame(table_data, columns=['Region', 'Target Variable', 'Model', 'RMSE', 'MAE'])
display(df)


folder_path = "C:/Users/vcostlea/OneDrive - NTT DATA EMEAL/Desktop/MFW/"
results_file = "results"
results_path = f"{folder_path}/Results/{results_file}"
index=False
header=True

df.to_excel(f"{results_path}.xlsx", index=index, header=header)
df.to_csv(f"{results_path}.csv", index=index, header=header)

print(f"File saved")
